In [1]:
#3
#athena db config - lifearchitectmodels.csv
#AAI-540 Group 3 FP

In [2]:
import boto3
import sagemaker
from pyathena import connect
import pandas as pd
sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name
ingest_create_athena_db_passed = False

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [3]:
database_name = "dsoaws"
s3_staging_dir = "s3://{0}/athena/staging".format(bucket)
conn = connect(region_name=region, s3_staging_dir=s3_staging_dir)

In [4]:
statement = "CREATE DATABASE IF NOT EXISTS {}".format(database_name)
print(statement)
pd.read_sql(statement, conn)

CREATE DATABASE IF NOT EXISTS dsoaws


/tmp/ipykernel_2982/2532319348.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(statement, conn)


""


In [5]:
statement = "SHOW DATABASES"
df_show = pd.read_sql(statement, conn)
df_show.head(5)

/tmp/ipykernel_2982/1677879827.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_show = pd.read_sql(statement, conn)


,database_name
0,default
1,dsoaws


In [6]:
s3_data_path = 's3://sagemaker-us-east-1-342942554664/lifearchitectmodels.csv'
s3_table_path = 's3://sagemaker-us-east-1-342942554664/table2/'

In [7]:
!aws s3 cp {s3_data_path} {s3_table_path}

copy: s3://sagemaker-us-east-1-342942554664/lifearchitectmodels.csv to s3://sagemaker-us-east-1-342942554664/table2/lifearchitectmodels.csv


In [8]:
#table2 - lifearchitectmodels.csv
table_name_csv = 'lifearchitectmodels'
conn = connect(region_name=region, s3_staging_dir=s3_staging_dir)
ingest_create_athena_table_csv_passed = False
dataexplore = pd.read_csv('lifearchitectmodels.csv')
dataexplore.head()

,Field,Achievement,Result,Human result,Outperforms human avg?,Model,Testing date,Extract
0,Music,97% of people can’t tell the difference betwee...,NaN,NaN,Yes,Multiple,Nov/2025,"""all participants were asked to listen to thre..."
1,Transcription,Transcribing handwritten historical documents.,99.44,96.0,Yes,Gemini 3,Nov/2025,"""The new Gemini model’s performance on HTR mee..."
2,Finance,Large Language Models pass CFA Level III.,79.10,50.0,Yes,o4-mini,Jul/2025,"""leading models demonstrate strong capabilitie..."
3,CBRN,LLMs can can accurately guide users through th...,NaN,NaN,Yes,GPT-4o,Jun/2025,"""we find that advanced AI models Llama 3.1 405..."
4,Health reviews,LLMs outperform humans in synthesizing results...,96.70,81.7,Yes,o3-mini-high,Jun/2025,"""We developed otto-SR, an end-to-end agentic w..."


In [9]:
dataexplore.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57 entries, 0 to 56
Data columns (total 8 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Field                   53 non-null     object 
 1   Achievement             55 non-null     object 
 2   Result                  24 non-null     float64
 3   Human result            12 non-null     float64
 4   Outperforms human avg?  53 non-null     object 
 5   Model                   54 non-null     object 
 6   Testing date            53 non-null     object 
 7   Extract                 53 non-null     object 
dtypes: float64(2), object(6)
memory usage: 3.7+ KB


In [10]:
!aws s3 cp s3://sagemaker-us-east-1-342942554664/lifearchitectmodels.csv \
           s3://sagemaker-us-east-1-342942554664/table2/

copy: s3://sagemaker-us-east-1-342942554664/lifearchitectmodels.csv to s3://sagemaker-us-east-1-342942554664/table2/lifearchitectmodels.csv


In [12]:
!aws s3 ls s3://sagemaker-us-east-1-342942554664/table2/

2026-02-12 08:52:09      18853 lifearchitectmodels.csv


In [13]:
#sql table creation
statement = """CREATE EXTERNAL TABLE IF NOT EXISTS {}.{}(
         Question STRING,
         Response STRING,
         Overall STRING,
         Age_18_34 STRING,
         Age_35_49 STRING,
         Age_50_64 STRING,
         Age_65_up STRING,
         Gender_Women STRING,
         Gender_Men STRING,
         College_No_College_College STRING,
         College_No_College_No_college STRING,
         Income_three_way_Lower_Income STRING,
         Income_three_way_Middle_Income STRING,
         Income_three_way_Upper_Income STRING,
         Generation_Gen_Z STRING,
         Generation_Millenial STRING,
         Generation_Gen_X STRING,
         Generation_Boomer STRING,
         Generation_Silent STRING,
         Race_Native_American STRING,
         Race_Asian STRING,
         Race_Hispanic STRING,
         Race_Black STRING,
         Race_White STRING,
         Occupation_Category_Office_Job STRING,
         Occupation_Category_Customer_Facing_Service_Job STRING,
         Occupation_Category_Manual_Not_Customer_Facing STRING,
         Occupation_Category_Education_Medicine STRING,
         Occupation_Category_Building_Trades STRING
) ROW FORMAT DELIMITED 
  FIELDS TERMINATED BY ',' 
  LINES TERMINATED BY '\\n' 
LOCATION '{}'
TBLPROPERTIES ('skip.header.line.count'='1')""".format(
    database_name, table_name_csv, s3_table_path
)

In [14]:
pd.read_sql(statement, conn)

/tmp/ipykernel_2982/618921652.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(statement, conn)


""


In [15]:
statement = "SHOW TABLES in {}".format(database_name)
df_show = pd.read_sql(statement, conn)
df_show.head(5)

/tmp/ipykernel_2982/84333453.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_show = pd.read_sql(statement, conn)


,tab_name
0,aimodelpoll
1,lifearchitectmodels


In [16]:
statement = """SELECT * FROM {}.{} LIMIT 10""".format(
    database_name, table_name_csv
)
print(statement)
df = pd.read_sql(statement, conn)
df.head()

SELECT * FROM dsoaws.lifearchitectmodels LIMIT 10


/tmp/ipykernel_2982/888292236.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(statement, conn)


,question,response,overall,age_18_34,age_35_49,age_50_64,age_65_up,gender_women,gender_men,college_no_college_college,...,race_native_american,race_asian,race_hispanic,race_black,race_white,occupation_category_office_job,occupation_category_customer_facing_service_job,occupation_category_manual_not_customer_facing,occupation_category_education_medicine,occupation_category_building_trades
0,Music,97% of people can’t tell the difference betwee...,,,Yes,Multiple,Nov/2025,"""""""all participants were asked to listen to th...",None,None,...,None,None,None,None,None,None,None,None,None,None
1,Transcription,Transcribing handwritten historical documents.,99.44,96.0,Yes,Gemini 3,Nov/2025,"""""""The new Gemini model’s performance on HTR m...",the new Gemini model was only getting about 1...,not counting punctuation marks and capital le...,...,None,None,None,None,None,None,None,None,None,None
2,Finance,Large Language Models pass CFA Level III.,79.1,50.0,Yes,o4-mini,Jul/2025,"""""""leading models demonstrate strong capabilities",with composite scores such as 79.1% (o4-mini)...,None,...,None,None,None,None,None,None,None,None,None,None
3,CBRN,LLMs can can accurately guide users through th...,,,Yes,GPT-4o,Jun/2025,"""""""we find that advanced AI models Llama 3.1 405B",ChatGPT-4o,and Claude 3.5 Sonnet can accurately guide us...,...,None,None,None,None,None,None,None,None,None,None
4,Health reviews,LLMs outperform humans in synthesizing results...,96.7,81.7,Yes,o3-mini-high,Jun/2025,"""""""We developed otto-SR",an end-to-end agentic workflow using large la...,97.9% specificity; human: 81.7% sensitivity,...,None,None,None,None,None,None,None,None,None,None
